In [7]:
from loader_util.dogs_vs_cats.config import dog_vs_cats_config as config
from loader_util.preprocessing import ImageToArrayPreprocessor, \
    SimplePreProcessor, MeanPreprocessor, CropPreprocessor
from loader_util.io import HDF5DatasetGenerator
from loader_util.utils.ranked import rank5_accuracy
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
import numpy as np
import progressbar
import json

In [8]:
# load the RGB means
means = json.loads(open(config.DATASET_MEAN).read())

# init the prepreocessors
sp = SimplePreProcessor(227, 227)
mp = MeanPreprocessor(rmean=means["R"],
                      gmean=means["G"],
                      bmean=means["B"])
cp = CropPreprocessor(227, 227)
iap = ImageToArrayPreprocessor()

In [9]:
# load the saved model
model = load_model(config.MODEL_PATH) # type: Model

In [10]:
# init the testing dataset generator then make predictions on the testing
# dataset
print(f'[INFO] predicting on test data with no crops.....')
test_gen = HDF5DatasetGenerator(dbPath=config.TEST_HDF5,
                                batchSize=64,
                                preprocessors=[sp, mp, iap],
                                classes=2)

predictions = model.predict_generator(test_gen.generator(),
                                      steps=test_gen.numImages//64,
                                      max_queue_size=10)
# compute the rank1 and rank5 accuracies
rank1, rank5 = rank5_accuracy(predictions, test_gen.db["labels"])
print(f'[INFO] rank1 accuracy : {rank1*100:0.2f}.....')

[INFO] predicting on test data with no crops.....
[INFO] rank1 accuracy : 89.38.....


In [12]:
# reinitialise test generator for cropping inclusion
# remmeber we only apply MC
test_gen2 = HDF5DatasetGenerator(config.TEST_HDF5,
                                 64,
                                 preprocessors=[mp],
                                 classes=2)

predictions = []

# init the progressbar
widgets = [" Evaluating:", " ",  progressbar.Percentage(), " ", progressbar
    .Bar(), " ", progressbar.ETA()]
pbar = progressbar.ProgressBar(maxval=test_gen.numImages // 64,
                               widgets=widgets).start()


# loop over a single image
for i, (images, labels) in enumerate(test_gen2.generator(passes=1)):
    for image in images:
        crops = cp.preprocess(image)
        crops = np.array([iap.preprocess(c) for c in crops], dtype="float")
        preds = model.predict(crops)
        predictions.append(preds.mean(axis=0))
    pbar.update(i)

pbar.finish()

 Evaluating: 100% |############################################| Time:  0:06:38


In [13]:
print(f'[INFO] predicting on test data with crops.....')
rank1, rank5 = rank5_accuracy(predictions, test_gen2.db["labels"])
print(f'[INFO] rank1: {rank1*100:0.2f}.....')
test_gen2.close()

[INFO] predicting on test data with crops.....
[INFO] rank1: 91.48.....
